<a href="https://colab.research.google.com/github/skrrtskrrt123/CPU-Scheduling-Simulator/blob/main/CPU_SchedulingSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import output
import IPython
from IPython.display import HTML, display
import json

class Process:
    def __init__(self, pid, arrival_time, burst_time, priority):
        self.pid = f"P{pid}"
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.priority = priority
        self.remaining_time = burst_time
        self.start_time = -1
        self.completion_time = 0
        self.turnaround_time = 0
        self.waiting_time = 0

def round_robin(processes, quantum=3):
    queue = []
    gantt_chart = []
    time = min(p.arrival_time for p in processes)
    processes.sort(key=lambda x: (x.arrival_time, x.priority, x.burst_time))
    remaining_processes = processes[:]

    queue.append(remaining_processes.pop(0))

    while queue:
        process = queue.pop(0)

        if process.start_time == -1:
            process.start_time = max(time, process.arrival_time)

        execution_time = min(quantum, process.remaining_time)
        gantt_chart.append((process.pid, time, time + execution_time))
        time += execution_time
        process.remaining_time -= execution_time

        for p in sorted(remaining_processes[:], key=lambda x: (x.arrival_time, x.priority, x.burst_time)):
            if p.arrival_time <= time:
                queue.append(p)
                remaining_processes.remove(p)

        if process.remaining_time > 0:
            queue.append(process)
        else:
            process.completion_time = time
            process.turnaround_time = process.completion_time - process.arrival_time
            process.waiting_time = process.turnaround_time - process.burst_time

    return [p for p in processes if p.completion_time > 0], gantt_chart

def sjn_scheduling(processes):
    processes_copy = [Process(p.pid.replace('P',''), p.arrival_time, p.burst_time, p.priority) for p in processes]
    current_time = 0
    completed_processes = []
    gantt_chart = []

    while processes_copy:
        available = [p for p in processes_copy if p.arrival_time <= current_time]

        if not available:
            current_time = min(p.arrival_time for p in processes_copy)
            continue

        selected = min(available, key=lambda x: x.burst_time)
        start_time = current_time
        current_time += selected.burst_time
        selected.completion_time = current_time
        selected.turnaround_time = current_time - selected.arrival_time
        selected.waiting_time = selected.turnaround_time - selected.burst_time

        gantt_chart.append((selected.pid, start_time, current_time))
        completed_processes.append(selected)
        processes_copy.remove(selected)

    return completed_processes, gantt_chart

def priority_nonpreemptive(processes):
    processes_copy = [Process(p.pid.replace('P',''), p.arrival_time, p.burst_time, p.priority) for p in processes]
    current_time = 0
    completed_processes = []
    gantt_chart = []

    while processes_copy:
        available = [p for p in processes_copy if p.arrival_time <= current_time]

        if not available:
            current_time = min(p.arrival_time for p in processes_copy)
            continue

        selected = min(available, key=lambda x: (x.priority, x.arrival_time))
        start_time = current_time
        current_time += selected.burst_time
        selected.completion_time = current_time
        selected.turnaround_time = current_time - selected.arrival_time
        selected.waiting_time = selected.turnaround_time - selected.burst_time

        gantt_chart.append((selected.pid, start_time, current_time))
        completed_processes.append(selected)
        processes_copy.remove(selected)

    return completed_processes, gantt_chart

def srt_scheduling(processes):
    processes_copy = [Process(p.pid.replace('P',''), p.arrival_time, p.burst_time, p.priority) for p in processes]
    current_time = 0
    completed_processes = []
    gantt_chart = []
    n = len(processes_copy)
    completed = 0
    last_process = None
    start_time = 0

    while completed != n:
        current_process = None
        for process in processes_copy:
            if (process.arrival_time <= current_time and process.remaining_time > 0 and
                    (current_process is None or process.remaining_time < current_process.remaining_time or
                     (process.remaining_time == current_process.remaining_time and
                      process.arrival_time < current_process.arrival_time))):
                current_process = process

        if current_process is None:
            current_time += 1
            continue

        if last_process != current_process:
            if last_process is not None:
                gantt_chart.append((last_process.pid, start_time, current_time))
            start_time = current_time
            last_process = current_process

        current_process.remaining_time -= 1
        current_time += 1

        if current_process.remaining_time == 0:
            completed += 1
            current_process.completion_time = current_time
            current_process.turnaround_time = current_time - current_process.arrival_time
            current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
            if current_process not in completed_processes:
                completed_processes.append(current_process)

    if last_process is not None:
        gantt_chart.append((last_process.pid, start_time, current_time))

    return completed_processes, gantt_chart

def process_data(data_str):
    try:
        data = json.loads(data_str)
        processes = [Process(p['pid'], p['arrival_time'], p['burst_time'], p['priority'])
                    for p in data['processes']]

        if data['algorithm'] == 'rr':
            completed_processes, gantt_chart = round_robin(processes, data['quantum'])
        elif data['algorithm'] == 'sjn':
            completed_processes, gantt_chart = sjn_scheduling(processes)
        elif data['algorithm'] == 'pnp':
            completed_processes, gantt_chart = priority_nonpreemptive(processes)
        else:  # srt
            completed_processes, gantt_chart = srt_scheduling(processes)

        result = {
            'processes': [{
                'pid': p.pid,
                'arrival_time': p.arrival_time,
                'burst_time': p.burst_time,
                'priority': p.priority,
                'completion_time': p.completion_time,
                'turnaround_time': p.turnaround_time,
                'waiting_time': p.waiting_time
            } for p in completed_processes],
            'gantt_chart': gantt_chart
        }

        return IPython.display.JSON(result)
    except Exception as e:
        return IPython.display.JSON({
            'error': str(e),
            'processes': [],
            'gantt_chart': []
        })

html_content = """<!DOCTYPE html>
<!DOCTYPE html>
<html>
<head>
    <style>
        .container {
            max-width: 1000px;
            margin: 0 auto;
            padding: 20px;
            font-family: Arial, sans-serif;
        }
        .input-section {
            background: #f5f5f5;
            padding: 20px;
            border-radius: 8px;
            margin-bottom: 20px;
        }
        .process-table {
            margin-top: 20px;
            width: 100%;
            border-collapse: collapse;
        }
        .process-table th, .process-table td {
            border: 1px solid #ddd;
            padding: 8px;
            text-align: center;
        }
        .process-table th {
            background-color: #f2f2f2;
            font-weight: bold;
        }
        .results {
            margin-top: 20px;
        }
        .summary-stats {
            margin-top: 20px;
            padding: 15px;
            background-color: #f8f9fa;
            border-radius: 5px;
        }
        button {
            background-color: #4CAF50;
            color: white;
            padding: 10px 20px;
            border: none;
            border-radius: 4px;
            cursor: pointer;
            margin: 10px 0;
        }
        button:hover {
            background-color: #45a049;
        }
        input[type="number"] {
            width: 80px;
            padding: 5px;
        }
        .algorithm-section {
            margin: 20px 0;
        }
        #ganttChart {
            margin-top: 20px;
            overflow-x: auto;
        }
        .hidden {
            display: none;
        }
        #processTable {
            display: none;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>CPU Scheduling Simulator</h1>

        <div class="input-section">
            <h3>Process Inputs</h3>
            <div class="process-controls">
                <label for="numProcesses">Number of Processes (3-10):</label>
                <input type="number" id="numProcesses" min="3" max="10" value="3">
                <button onclick="generateProcessInputs()">Generate Process Fields</button>
            </div>

            <div id="processTable">
                <table class="process-table">
                    <thead>
                        <tr>
                            <th>Process ID</th>
                            <th>Arrival Time</th>
                            <th>Burst Time</th>
                            <th>Priority</th>
                        </tr>
                    </thead>
                    <tbody id="processInputs">
                    </tbody>
                </table>
            </div>
        </div>

        <div class="algorithm-section">
            <h3>Select Algorithm:</h3>
            <input type="radio" name="algorithm" value="rr" checked> Round Robin
            <input type="number" id="quantum" value="3" min="1" style="width: 50px;"> (Quantum)
            <br>
            <input type="radio" name="algorithm" value="sjn"> Shortest Job Next
            <br>
            <input type="radio" name="algorithm" value="pnp"> Priority (Non-preemptive)
            <br>
            <input type="radio" name="algorithm" value="srt"> Shortest Remaining Time
        </div>

        <button onclick="runSimulation()">Run Simulation</button>

        <div class="results" id="results">
            <div id="ganttChart"></div>
        </div>
    </div>

    <script>
        function generateProcessInputs() {
            const numProcesses = parseInt(document.getElementById('numProcesses').value);
            const tbody = document.getElementById('processInputs');
            const processTable = document.getElementById('processTable');

            tbody.innerHTML = '';

            for (let i = 0; i < numProcesses; i++) {
                const row = document.createElement('tr');
                row.innerHTML = `
                    <td>P${i}</td>
                    <td><input type="number" min="0" value="0" id="arrival${i}" class="process-input"></td>
                    <td><input type="number" min="1" value="1" id="burst${i}" class="process-input"></td>
                    <td><input type="number" min="0" value="0" id="priority${i}" class="process-input"></td>
                `;
                tbody.appendChild(row);
            }

            processTable.style.display = 'block';
        }

        function collectProcessData() {
            const processes = [];
            const numProcesses = parseInt(document.getElementById('numProcesses').value);

            for (let i = 0; i < numProcesses; i++) {
                processes.push({
                    pid: i,
                    arrival_time: parseInt(document.getElementById(`arrival${i}`).value),
                    burst_time: parseInt(document.getElementById(`burst${i}`).value),
                    priority: parseInt(document.getElementById(`priority${i}`).value)
                });
            }
            return processes;
        }

        function drawGanttChart(ganttData) {
            const container = document.getElementById('ganttChart');
            container.innerHTML = '<h3>Gantt Chart</h3>';

            const chart = document.createElement('div');
            chart.style.display = 'flex';
            chart.style.marginTop = '10px';
            chart.style.position = 'relative';
            chart.style.height = '80px';

            let maxTime = 0;
            ganttData.forEach(([_, __, end]) => maxTime = Math.max(maxTime, end));

            const wrapper = document.createElement('div');
            wrapper.style.position = 'relative';
            wrapper.style.width = '100%';
            wrapper.style.marginBottom = '30px';

            ganttData.forEach(([process, start, end]) => {
                const block = document.createElement('div');
                block.style.position = 'absolute';
                block.style.left = `${(start/maxTime) * 100}%`;
                block.style.width = `${((end-start)/maxTime) * 100}%`;
                block.style.height = '40px';
                block.style.backgroundColor = '#add8e6';
                block.style.border = '1px solid #666';
                block.style.display = 'flex';
                block.style.alignItems = 'center';
                block.style.justifyContent = 'center';
                block.style.fontSize = '14px';
                block.innerHTML = `${process}`;

                const startLabel = document.createElement('div');
                startLabel.style.position = 'absolute';
                startLabel.style.bottom = '-25px';
                startLabel.style.left = '0';
                startLabel.style.transform = 'translateX(-50%)';
                startLabel.innerHTML = start;
                block.appendChild(startLabel);

                if (end !== maxTime) {
                    const endLabel = document.createElement('div');
                    endLabel.style.position = 'absolute';
                    endLabel.style.bottom = '-25px';
                    endLabel.style.right = '0';
                    endLabel.style.transform = 'translateX(50%)';
                    endLabel.innerHTML = end;
                    block.appendChild(endLabel);
                }

                chart.appendChild(block);
            });

            const finalMarker = document.createElement('div');
            finalMarker.style.position = 'absolute';
            finalMarker.style.bottom = '-25px';
            finalMarker.style.right = '0';
            finalMarker.style.transform = 'translateX(50%)';
            finalMarker.innerHTML = maxTime;
            chart.appendChild(finalMarker);

            wrapper.appendChild(chart);
            container.appendChild(wrapper);
        }

        function displayResults(processes, ganttData) {
            const resultsDiv = document.getElementById('results');
            resultsDiv.innerHTML = '<div id="ganttChart"></div>';

            drawGanttChart(ganttData);

            const metricsSection = document.createElement('div');
            metricsSection.className = 'metrics-section';
            metricsSection.innerHTML = `
                <h3>Process Metrics</h3>
                <table class="process-table">
                    <thead>
                        <tr>
                            <th>Process</th>
                            <th>Arrival Time</th>
                            <th>Burst Time</th>
                            <th>Priority</th>
                            <th>Completion Time</th>
                            <th>Turnaround Time</th>
                            <th>Waiting Time</th>
                        </tr>
                    </thead>
                    <tbody>
                        ${processes.map(p => `
                            <tr>
                                <td>${p.pid}</td>
                                <td>${p.arrival_time}</td>
                                <td>${p.burst_time}</td>
                                <td>${p.priority}</td>
                                <td>${p.completion_time}</td>
                                <td>${p.turnaround_time}</td>
                                <td>${p.waiting_time}</td>
                            </tr>
                        `).join('')}
                    </tbody>
                </table>
            `;
            resultsDiv.appendChild(metricsSection);

            const avgTurnaround = processes.reduce((sum, p) => sum + p.turnaround_time, 0) / processes.length;
            const avgWaiting = processes.reduce((sum, p) => sum + p.waiting_time, 0) / processes.length;

            const summaryStats = document.createElement('div');
            summaryStats.className = 'summary-stats';
            summaryStats.innerHTML = `
                <h3>Summary Statistics</h3>
                <table class="process-table">
                    <tr>
                        <th>Metric</th>
                        <th>Value</th>
                    </tr>
                    <tr>
                        <td>Average Turnaround Time</td>
                        <td>${avgTurnaround.toFixed(2)}</td>
                    </tr>
                    <tr>
                        <td>Average Waiting Time</td>
                        <td>${avgWaiting.toFixed(2)}</td>
                    </tr>
                </table>
            `;
            resultsDiv.appendChild(summaryStats);
        }

        async function runSimulation() {
            try {
                const processes = collectProcessData();
                const algorithm = document.querySelector('input[name="algorithm"]:checked').value;
                const quantum = parseInt(document.getElementById('quantum').value);

                const data = {
                    processes: processes,
                    algorithm: algorithm,
                    quantum: quantum
                };

                console.log('Sending data to Python:', data);

                const result = await google.colab.kernel.invokeFunction(
                    'process_data',
                    [JSON.stringify(data)],
                    {}
                );

                if (!result.data || !result.data['application/json']) {
                    throw new Error('Invalid response format');
                }

                const output = result.data['application/json'];
                console.log('Parsed output:', output);

                if (!output.processes || !output.gantt_chart) {
                    throw new Error('Missing required data in output');
                }

                displayResults(output.processes, output.gantt_chart);
            } catch (error) {
                console.error('Error in simulation:', error);
                document.getElementById('results').innerHTML = `
                    <div style="color: red; padding: 10px; border: 1px solid red; margin: 10px 0;">
                        Error running simulation. Please check your inputs and try again.
                        Technical details: ${error.message}
                    </div>
                `;
            }
        }

        // Initialize the page
        generateProcessInputs();
    </script>
</body>
</html>
"""


def process_data(data_str):
    try:
        data = json.loads(data_str)
        processes = [Process(p['pid'], p['arrival_time'], p['burst_time'], p['priority'])
                    for p in data['processes']]

        if data['algorithm'] == 'rr':
            completed_processes, gantt_chart = round_robin(processes, data['quantum'])
        elif data['algorithm'] == 'sjn':
            completed_processes, gantt_chart = sjn_scheduling(processes)
        elif data['algorithm'] == 'pnp':
            completed_processes, gantt_chart = priority_nonpreemptive(processes)
        else:  # srt
            completed_processes, gantt_chart = srt_scheduling(processes)

        result = {
            'processes': [{
                'pid': p.pid,
                'arrival_time': p.arrival_time,
                'burst_time': p.burst_time,
                'priority': p.priority,
                'completion_time': p.completion_time,
                'turnaround_time': p.turnaround_time,
                'waiting_time': p.waiting_time
            } for p in completed_processes],
            'gantt_chart': gantt_chart
        }

        return IPython.display.JSON(result)
    except Exception as e:
        return IPython.display.JSON({
            'error': str(e),
            'processes': [],
            'gantt_chart': []
        })

# Register the callback
output.register_callback('process_data', process_data)

# Display the HTML interface
display(HTML(html_content))